In [1]:
import numpy,csv,os,re

In [ ]:
import csv #导出CSV文件（点位+路段）信息
section_dict={}
info=[0,2,6,8,24,44,45]
with open(r'..\路段信息.csv','r',errors='ignore',encoding='utf-8-sig') as file:
    rows=csv.reader(file)
    for index,row in enumerate(rows):
        if index>0:
            section_id=row[1]
            section_dict[section_id]=[row[each] for each in info]
numpy.save(r'..\Python Scripts\40天统计信息及路段情况\路段信息.npy',section_dict)

In [ ]:
# #查看天气
# rainy_dates=[]
# with open(r'C:\Users\e-wel\Desktop\2018.5.txt','r',errors='ignore',encoding='utf-8') as status_file:
#     line_list=status_file.readlines()
#     for line in line_list:
#         raw_record=line.split(' ')[:2]
#         obj=re.search('(\d*)月(\d*)日',raw_record[0]).group(1,2)
#         date=obj[0]+obj[1]
#         if ('大雨' or '中雨') in raw_record[1]:
#             rainy_dates.append(date)
# print(rainy_dates) 

In [ ]:
kakou=numpy.load(r'..\卡口点位信息.npy',allow_pickle=True).item()

In [ ]:
segment=numpy.load(r'..\路段信息_简略.npy',allow_pickle=True).item()

In [ ]:
area_dict={}
for kk,value in kakou.items():
    if value[2] not in area_dict:
        area_dict[value[2]]=[]
    area_dict[value[2]].append([kk,value[1]])

In [ ]:
matched={}#点位路段匹配
for seg,value in segment.items():
    dianwei_area=area_dict[value[5]]
    location=value[4].split(',')
    location=[float(each) for each in location]
    weidus=[]
    jindus=[]
    i=0
    while i < len(location)-1:
        jindus.append(location[i])
        weidus.append(location[i+1])
        i+=2
    weidu_max=max(weidus)
    jindu_max=max(jindus)
    weidu_min=min(weidus)
    jindu_min=min(jindus)
    dianwei_list=[]
    for dianwei in dianwei_area:
        j,w=dianwei[1]
        j=float(j)
        w=float(w)
        if j<jindu_max and j>jindu_min and w<weidu_max and w>weidu_min:
            # matched[dianwei[0]]=seg
            dianwei_list.append(dianwei[0])
    segment[seg].append(dianwei_list)
            

In [ ]:
numpy.save(r'..\Python Scripts\60天统计\点位_to_路段.npy',matched)
numpy.save(r'..\Python Scripts\60天统计\区域_to_点位.npy',area_dict)

In [ ]:
for seg,value in matched.items():
    if len(value)>1:
        print(seg,len(value))

In [ ]:
time_stamp=numpy.load(r'..\Python Scripts\40天原始记录\time_stamp.npy',allow_pickle=True).item()
section_dict=numpy.load(r'..\Python Scripts\60天统计\路段信息.npy',allow_pickle=True).item()
matched=numpy.load(r'..\Python Scripts\60天统计\点位_to_路段.npy',allow_pickle=True).item()

In [ ]:
# 导出旅行时间记录
def dump_out(date,txt_name):
    '''
    date:日期，txt_name:原始数据文件名称
    '''
    save_root=r'..\Python Scripts\5-6月记录'
    date_dict={}#section_id:{timestamp_0:value}
    with open(r'..\人工智慧\交通数据\2018.08-09卡口状态数据\{}'.format(txt_name),'r',errors='ignore',encoding='utf-8') as status_file:
        line_list=status_file.readlines()
        for line in line_list:
            raw_record=line.split(',')
            section_id=int(section_dict[raw_record[1]][0])
            if section_id not in date_dict:
                date_dict[section_id]=time_stamp.copy()
            time_stamp_str=re.search('(\d*:\d*):\d*',raw_record[4]).group(1)
            hour=int(time_stamp_str[0:2])
            if hour<23 and hour>7:#选择所要研究的时间段
                date_dict[section_id][time_stamp_str]=raw_record[3]
    save_path=os.path.join(save_root,date,date+'_raw_data.npy')
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    numpy.save(save_path,date_dict)        


In [ ]:
# 导出流量记录
def dump_out_flow(date,txt_name):
    '''
    date:日期，txt_name:原始数据文件名称
    '''
    save_root=r'C:\Users\e-wel\Documents\Python Scripts\5-6月记录'
    date_dict={}#section_id:{timestamp_0:value}
    with open(r'C:\Users\e-wel\Documents\人工智慧\交通数据\2018.08-09卡口流量数据\5min\{}'.format(txt_name),'r',errors='ignore',encoding='utf-8') as status_file:
        line_list=status_file.readlines()
        for line in line_list:
            raw_record=line.split(',')
            dianwei_id=raw_record[0]
            if dianwei_id not in matched:
                continue
            section_id=matched[dianwei_id]
            if section_id not in date_dict:
                date_dict[section_id]={}
            time_stamp_str=re.search('(\d*:\d*):\d*',raw_record[5]).group(1)
            hour=int(time_stamp_str[0:2])
            if hour<23 and hour>7:
                if time_stamp_str not in date_dict[section_id]:
                    date_dict[section_id][time_stamp_str]=[[],[]]
                date_dict[section_id][time_stamp_str][0].append(raw_record[4])
                date_dict[section_id][time_stamp_str][1].append(raw_record[2])
    save_path=os.path.join(save_root,date,date+'_flow.npy')
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    numpy.save(save_path,date_dict)        

In [ ]:
dump_out('0603','SECTION_STATUS_COMPUTE_20180603_033229.txt')#使用函数的示例

In [ ]:
# 从原始交通记录中导出目标格式下的数据
for file in os.listdir(r'..\人工智慧\交通数据\2018.08-09卡口流量数据\5min')[1:]:
    obj=re.search('2018(\d*)',file)
    if obj is not None:
        date=obj.group(1)
        dump_out_flow(date,file)
        print(date)

In [ ]:
d1=numpy.load('导出数据.npy',allow_pickle=True).item()
new_dict={}
for seg,value in d1.items():
    new_dict[seg]=[]
    for day_record in value:
        for index in range(900):
            if day_record[index]=='':#将缺失值用0补充
                day_record[index]=0
            else:
                day_record[index]=int(day_record[index])#如果是用用整数的形式导出原始记录，此步可以忽略
        new_dict[seg].append(day_record)

In [5]:
min_tt=numpy.load(r'..\Python Scripts\40天统计信息及路段情况\min_travel_time.npy',allow_pickle=True).item()

In [ ]:
# 用上一时间点的值填补当下的缺失值
for seg,records in new_dict.items():
    for day_index,day in enumerate(records):
        for index,each in enumerate(day):
            if each==0:
                if index==0:#如果一天中第一个时间点的值缺失，用该路段的最短旅行时间填补
                    day[0]=min_tt[str(seg)]
                else:
                    day[index]=day[index-1]
        records[day_index]=day
    new_dict[seg]=records 

In [78]:
#整合为5分钟间隔的记录
for seg,records in new_dict.items():
    for day_index,day in enumerate(records):
        index=0
        new_day=[]
        while index<900:
            new_day.append(numpy.around(numpy.mean(day[index:index+5]),2))
            index+=5
        records[day_index]=new_day
    new_dict[seg]=records
